In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, f1_score, precision_score, recall_score, log_loss

In [ ]:
from explainerdashboard.explainers import *
from explainerdashboard.dashboards import *
from explainerdashboard.datasets import *

In [ ]:
feature_descriptions = {
    "Sex": "Gender of passenger",
    "Deck": "The deck the passenger had their cabin on",
    "PassengerClass": "The class of the ticket: 1st, 2nd or 3rd class",
    "Fare": "The amount of money people paid", 
    "No_of_relatives_on_board": "number of siblings, spouses, parents plus children on board",
    "Embarked": "the port where the passenger boarded the Titanic. Either Southampton, Cherbourg or Queenstown",
    "Age": "Age of the passenger",
    "No_of_siblings_plus_spouses_on_board": "The sum of the number of siblings plus the number of spouses on board",
    "No_of_parents_plus_children_on_board" : "The sum of the number of parents plus the number of children on board",
}

# ClassifierBunch example

## Load classifier data:
    - predicting probability that a person on the titanic survived

In [ ]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

## RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestClassifierExplainer(model, X_test, y_test, metric=roc_auc_score, 
                               shap='tree',
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               descriptions=feature_descriptions,
                               labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        decision_trees=True)
db.run(8052)

## LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

explainer = ClassifierExplainer(model, X_test, y_test, metric=roc_auc_score, 
                               shap='linear',
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               descriptions=feature_descriptions,
                               labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False, # Linear models have no interactions
                        decision_trees=False) # Linear models have no decision trees
db.run(8052)

## XGBClassifier

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

explainer = ClassifierBunch(model, X_test, y_test, roc_auc_score, shap='tree',
                                   cats=['Sex', 'Deck', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])



In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) 
db.run(8052)

## LGBMClassifier

In [ ]:
from lightgbm.sklearn import LGBMClassifier

model = LGBMClassifier()
model.fit(X_train, y_train)

explainer = ClassifierBunch(model, X_test, y_test, roc_auc_score, shap='tree',
                                   cats=['Sex', 'Deck', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) #only supported for output="raw" 
db.run(8052)

## CatBoostClassifier

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=100, learning_rate=100)

model.fit(X_train, y_train)

explainer = ClassifierBunch(model, X_test, y_test, roc_auc_score, shap='tree',
                                   cats=['Sex', 'Deck', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) 
db.run(8052)

# RegressionBunch example

## Load regression data:
    - predicting the fare that a titanic passenger paid for their ticket

In [ ]:
X_train, y_train, X_test, y_test = titanic_fare()
train_names, test_names = titanic_names()

## RandomForestRegressor

In [ ]:
model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestRegressionBunch(model, X_test, y_test, r2_score,
                shap='tree', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        decision_trees=True)
db.run(8052)

## LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='linear', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        §contributions=True,
                        shap_dependence=True,
                        shap_interaction=False)
db.run(8052)

## XGBRegressor

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='linear', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) #only model_output = "raw" supportted with interaction values for now
db.run(8052)

## LightGBM

In [ ]:
from lightgbm.sklearn import LGBMRegressor

model = LGBMRegressor()
model.fit(X_train, y_train)

explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='tree', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True) 
db.run(8052)

## CatBoost

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=100, learning_rate=0.1, verbose=0)
model.fit(X_train, y_train)

explainer = RegressionBunch(model, X_test, y_test, r2_score, 
                shap='tree', cats=['Sex', 'Deck', 'Embarked'], idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False) # interactions not supported for now due to this issue: https://github.com/slundberg/shap/issues/480 
db.run(8052)